#### **Quantum Explorers 2023**
---
# Time Traveler Badge - Quantum Error Correction & Mitigation Module
#### **Advanced Syllabus - Quantum Error Mitigation Methods and Implementation**
Adapted from [IBM Quantum Challenge Fall 2022, Lab 1](https://github.com/qiskit-community/ibm-quantum-challenge-fall-22/tree/main/content/lab-1)

### T-REX

T-REX stands for **Twirled Readout Error eXtinction** which is an implementation which involves "twirling" of gates. If we were to view noise as a set of extra probabilistic gates on top of our perfect circuit implementation, twirling is something that can be considered in the sense that every time we execute the circuit, we conjugate this noisy gate set with a gate randomly chosen from a set of gates called as a "twirling set". If we are to choose this gate set to be a set of Pauli operators, we can term this as "Pauli twirling: which is one of the most commonly used twirling techniques available.

Pauli twirling is a form of randomized compiling that inserts pairs of Pauli gates (I, X, Y, Z) before and after entangling gates such that the overall unitary is the same, but the way it is implemented is different. This has the effect of turning coherent errors into stochastic errors, which can then be eliminated by sufficient averaging. This is done a number of times for the benefit of averaging. Note: we may not use a sufficient basis set to currently cancel all errors. **[[1]](https://github.com/qiskit-research/qiskit-research/blob/main/docs/getting_started.ipynb)**

For more information on Pauli Twirling, watch [this Nick Knows video](https://www.youtube.com/watch?v=4MLHvmmpSQ8).

You can use activate T-REX on Estimator by setting `resilience_level=1` in options. T-REX will be active by default on runtime if no `resilience_level` is being specified. You may notice with Qiskit Runtime that the results may change every run when selecting T-REX. This is because the gate set selected for twirling is different everytime which contributes to the random characteristics observed in the result.

You can read more about T-REX [here](https://qiskit.org/ecosystem/ibm-runtime/tutorials/Error-Suppression-and-Error-Mitigation.html#Twirled-readout-error-extinction).

### Digital ZNE

Digital Zero Noise Extrapolation (ZNE) is a popular technique for mitigating errors in noisy quantum computers without the need for additional quantum resources. A quantum program is altered to run at different effect levels of processor noise. The result of the computation is extrapolated to an estimated value at a noiseless level.

To find the proper value, Digital ZNE firstly scales noise as explained above. There are two kinds of methods for scaling it: analog and digital. You can check the detailed description of each method below. It is still an active research question around which method is the best to use. Note that the digital way is by not acting on the physical quantum pulses.

- Pulse stretching (analog): Applying the same one stretched along a larger amount of time for a circuit - take physical pulses and stretch it in time. In principle its the same circuit - but you are increasing the effective noise. **[[2]](https://arxiv.org/pdf/1612.02058.pdf)**
- Local folding (digital): Compiling the input circuit with a larger number of gates. Each gate $G$ is replaced by $G$, $G^\dagger$, $G$. If you are on a simulator, you do nothing to the circuit. If you use a real device, you can see the increasing noise.
- Global folding (digital): Working the same way as local folding - but instead you apply this trick to the full circuit.

The noisy expectation values are then extrapolated to the zero-noise limit. This is generally done with general regression methods. 

You can read more about Digital ZNE [here](https://qiskit.org/ecosystem/ibm-runtime/tutorials/Error-Suppression-and-Error-Mitigation.html#Zero-noise-extrapolation).


### PEC

Probabilistic Error Cancellation (PEC) is an approach that mitigates error by learning and inverting a sparse noise model that is able to capture correlated noise. PEC returns an unbiased estimate of an expectation value so long as learned noise model faithfully represents the actual noise model at the time of mitigation. 

PEC uses a quasi-probability method to mimic the effect of inverting the learned noise. This requires sampling from a randomized circuit family associated with the user’s original circuit. Applying PEC will increase the variability of the returned expectation value estimates unless the number of samples per circuit is also increased for both input and characterization circuits. The amount of samples required to counter this variability scales exponentially with the noise strength of the mitigated circuit. **[[3]](https://qiskit.org/ecosystem/ibm-runtime/how_to/error-mitigation.html)**

You can read more about PEC [here](https://qiskit.org/ecosystem/ibm-runtime/tutorials/Error-Suppression-and-Error-Mitigation.html#Probabilistic-error-cancellation).


<div class="alert alert-block alert-info">
    
<h3>Implementation of Quantum Error Mitigation Techniques</h3>

The resilience level in Qiskit runtime is a metric to specify how much resilience to build against errors, and therefore the level of error mitigation. Higher levels generate more accurate results, at the expense of longer processing times. You can activate higher levels of resilience on Qiskit runtime but please do note that **this feature is currently in beta** and the results may not be as expected, but feel free to experiment with them where you are directed below in the code blocks!

You may find these resources handy:

- [Qiskit Documentation: Options](https://qiskit.org/ecosystem/ibm-runtime/stubs/qiskit_ibm_runtime.options.Options.html)
- [Configuring error mitigation](https://qiskit.org/ecosystem/ibm-runtime/how_to/error-mitigation.html)
    


In [ ]:
#import required libraries
import time
import numpy as np
from qiskit import *
from qiskit.circuit import Parameter
from qiskit.quantum_info import Statevector, Pauli, SparsePauliOp
from qiskit.circuit.library import RealAmplitudes
import matplotlib.pyplot as plt
import matplotlib.ticker as tck
from qiskit.tools.visualization import plot_histogram
from qiskit_ibm_runtime import QiskitRuntimeService, Session, Options, Sampler, Estimator
from qiskit.providers.fake_provider import FakeManila
from qiskit_aer.noise import NoiseModel

# Import FakeBackend
fake_backend = FakeManila()
noise_model = NoiseModel.from_backend(fake_backend)

In [ ]:
# Save the Runtime account credentials if you have not done so already
# If you need to overwrite the account info, please add `overwrite=True`
# QiskitRuntimeService.save_account(channel='ibm_quantum', token='my_token', overwrite=True)

service = QiskitRuntimeService(channel="ibm_quantum", instance='ibm-q/open/main')

backend = service.backends(simulator=True)[0]
print(backend)

In [ ]:
# This is the circuit you used in the main syllabus for learning M3
theta = Parameter('theta')

qc = QuantumCircuit(2)
qc.x(1)
qc.h(0)
qc.cp(theta,0,1)
qc.h(0)

qc.draw()

In the next cell, in the line of code as indicated, change the resilience level to any of the values `[1, 2, 3]` and run the Estimator to see how output changes with resilience level

In [ ]:
phases = np.linspace(0, 2*np.pi, 50)
individual_phases = [[ph] for ph in phases]
ZZ = SparsePauliOp.from_list([("ZZ", 1)])

options = Options(
    simulator={
        "noise_model": noise_model,
        "seed_simulator": 42,
    },  
    resilience_level=0
)

options_with_em = Options(
    simulator={
        "noise_model": noise_model,
        "seed_simulator": 42,
    },  
    resilience_level=1 ### Change the value in this line ###
)

In [ ]:
with Session(service=service, backend=backend):    
    estimator = Estimator(options=options)
    job = estimator.run(circuits=[qc]*len(phases), parameter_values=individual_phases, observables=[ZZ]*len(phases))
    param_results = job.result()
    exp_values = param_results.values
    
    estimator = Estimator(options=options_with_em)
    job = estimator.run(circuits=[qc]*len(phases), parameter_values=individual_phases, observables=[ZZ]*len(phases))
    param_results = job.result()
    exp_values_with_em = param_results.values

In [ ]:
plt.plot(phases, exp_values, 'o', label='noisy')
plt.plot(phases, exp_values_with_em, 'o', label='mitigated')
plt.plot(phases, 2*np.sin(phases/2,)**2-1, label='theory')
plt.xlabel('Phase')
plt.ylabel('Expectation')
plt.legend();

<div class="alert alert-block alert-info">
    
<h3>Optional: Run the above on a real backend</h3>

Feel free to test the above code on a **real quantum backend** to see how the result varies when running on a QPU. For Quantum Explorers, you have a `hub`, `group`, and `project` allocated to you. Feel free to configure the cell below and experiment with the results!
    
</div> 

In [ ]:
# Get the least-busy backend, this step may take a while
real_backend = service.least_busy(min_num_qubits=2, simulator=False)

print(f"The least busy backend is {real_backend.name}.")

Using the real backend for `Estimator`:

In [ ]:
with Session(service=service, backend=real_backend):    
    estimator = Estimator(options=options)
    job = estimator.run(circuits=[qc]*len(phases), parameter_values=individual_phases, observables=[ZZ]*len(phases))
    print(f"First job ID: {job.job_id()}")
    
    estimator = Estimator(options=options_with_em)
    job = estimator.run(circuits=[qc]*len(phases), parameter_values=individual_phases, observables=[ZZ]*len(phases))
    print(f"Second job ID: {job.job_id()}")

In [ ]:
#Get job results
job = service.job('place first job ID here')
param_results = job.result()
exp_values = param_results.values

job_with_em = service.job('place second job ID here')
param_results_with_em = job_with_em.result()
exp_values_with_em = param_results_with_em.values

While your jobs are running, you can monitor them! Simply copy the Job ID printed out for you for one of the runs and enter it into the search bar **[here](https://quantum-computing.ibm.com/jobs)**! Make sure you are logged into the IBM Quantum Platform to access this page. 

In [ ]:
plt.plot(phases, exp_values, 'o', label='noisy')
plt.plot(phases, exp_values_with_em, 'o', label='mitigated')
plt.plot(phases, 2*np.sin(phases/2,)**2-1, label='theory')
plt.xlabel('Phase')
plt.ylabel('Expectation')
plt.legend();

# References
- [1] **[Getting Started - Qiskit Research](https://github.com/qiskit-research/qiskit-research/blob/main/docs/getting_started.ipynb)**
- [2] **[Error mitigation for short-depth quantum circuits - Kristan Temme, Sergey Bravyi and Jay M. Gambetta](https://arxiv.org/pdf/1612.02058.pdf)**
- [3] **[Configure error mitigation](https://qiskit.org/ecosystem/ibm-runtime/how_to/error-mitigation.html)**

In [1]:
from qiskit.tools.jupyter import *
%qiskit_version_table